# Testing colmap for bundle adjustment

In [1]:
import pycolmap
import os
import numpy as np
from scipy.spatial.transform import Rotation as R

# Define input and output directories
out_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/ID-MCS/20240420/colmap'
image_dir = os.path.join(out_dir, 'images')

os.makedirs(out_dir, exist_ok = True)
mvs_path = os.path.join(out_dir, "mvs")
database_path = os.path.join(out_dir, "database.db")

In [ ]:
# Run feature extraction, feature matching, and global bundle adjustment
pycolmap.extract_features(database_path, image_dir)
pycolmap.match_exhaustive(database_path)
recons = pycolmap.incremental_mapping(database_path, image_dir, out_dir)

# Save reconstructions to file
for i in range(0,len(recons)):
    recon_dir = os.path.join(out_dir, f"reconstruction_{i}")
    os.makedirs(recon_dir, exist_ok=True)
    recons[i].write(recon_dir)

W20250905 10:01:31.023273 0x7ff844818300 feature_extraction.cc:404] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20250905 10:01:31.044149 0x7000152a6000 misc.cc:44] 
Feature extraction
I20250905 10:01:31.044505 0x700015b59000 sift.cc:727] Creating SIFT CPU feature extractor
I20250905 10:01:31.044547 0x700015bdc000 sift.cc:727] Creating SIFT CPU feature extractor
I20250905 10:01:31.044621 0x700015d65000 sift.cc:727] Creating SIFT CPU feature extractor
I20250905 10:01:31.044628 0x700015e6b000 sift.cc:727] Creating SIFT CPU feature extractor
I20250905 10:01:31.044653 0x700015ce2000 sift.cc:727] Creating SIFT CPU feature extractor
I20250905 10:01:31.044667 

AttributeError: 'int' object has no attribute 'write'

In [10]:
# Save reconstructions to file
for i in range(0,len(recons)):
    recon_dir = os.path.join(out_dir, f"reconstruction_{i}")
    os.makedirs(recon_dir, exist_ok=True)
    recons[i].write(recon_dir)

In [ ]:
# Convert camera models to TXT files
# import subprocess

# cmd = ['colmap', 'model_converter',
#        '--input_path', out_dir,
#        '--output_path', out_dir,
#        '--output_type', 'TXT']

# subprocess.run(cmd)

In [ ]:
# Convert COLMAP cameras to TSAI format for use in ASP
def colmap_to_tsai(cam, img, output_path):
    """
    cam: COLMAP camera object (intrinsics)
    img: COLMAP image object (extrinsics)
    output_path: path to save .tsai
    """

    # Intrinsics
    fx, fy, cx, cy = cam.params  # assuming PINHOLE model
    width, height = cam.width, cam.height

    # Extrinsics from COLMAP
    q = img.qvec  # [qw, qx, qy, qz]
    t = img.tvec  # translation
    R_wc = R.from_quat([q[1], q[2], q[3], q[0]]).as_matrix()  # world->cam
    C = -R_wc.T @ t  # camera center in world coords

    # Convert to ASP convention
    R_cw = R_wc.T  # camera->world

    with open(output_path, "w") as f:
        f.write("VERSION_4\n")
        f.write("PINHOLE\n")
        f.write(f"fu = {fx}\n")
        f.write(f"fv = {fy}\n")
        f.write(f"cu = {cx}\n")
        f.write(f"cv = {cy}\n")
        f.write("u_direction = 1 0 0\n")
        f.write("v_direction = 0 1 0\n")
        f.write("w_direction = 0 0 1\n")
        f.write(f"C = {C[0]} {C[1]} {C[2]}\n")
        f.write("R = " + " ".join(map(str, R_cw.flatten())) + "\n")
        f.write("pitch = 0.8\n")
        f.write("NULL\n")


recon = pycolmap.Reconstruction(out_dir)
for img_id, img in recon.images.items():
    cam = recon.cameras[img.camera_id]
    out_tsai = f"{img.name}.tsai"
    colmap_to_tsai(cam, img, out_tsai)
